In [1]:
import numpy as np

IDSet = set(np.load("/home/wngys/lab/DeepFold/protein_infor/IDArray.npy", allow_pickle=True).tolist())
print(len(IDSet))
print(list(IDSet)[:10])

14274
['d5u89b_', 'd2ioja1', 'd2a2ra2', 'd1na6a1', 'd5fxda2', 'd2vg0a_', 'd1vj1a1', 'd3cx5f_', 'd1ufwa1', 'd1qcxa_']


In [2]:
errorPathList = np.load("/home/wngys/lab/DeepFold/protein_infor/entPathArrayResError.npy", allow_pickle=True).tolist()
print(len(errorPathList))
print(errorPathList)

12
['/home/wngys/scop/scope-2.07-40/de/d1deua_.ent', '/home/wngys/scop/scope-2.07-40/ng/d1ngvb_.ent', '/home/wngys/scop/scope-2.07-40/bd/d1bd7a_.ent', '/home/wngys/scop/scope-2.07-40/nr/d1nria_.ent', '/home/wngys/scop/scope-2.07-40/eu/d1eu3a1.ent', '/home/wngys/scop/scope-2.07-40/ms/d3msza1.ent', '/home/wngys/scop/scope-2.07-40/pc/d1pcia_.ent', '/home/wngys/scop/scope-2.07-40/mh/d1mh1a_.ent', '/home/wngys/scop/scope-2.07-40/no/d1nova_.ent', '/home/wngys/scop/scope-2.07-40/pv/d1pvta_.ent', '/home/wngys/scop/scope-2.07-40/ax/d4ax1b_.ent', '/home/wngys/scop/scope-2.07-40/dk/d1dkia_.ent']


In [3]:
errorIDSet = set([ele.split('/')[-1].split('.')[0] for ele in errorPathList])
print(len(errorIDSet))
print(errorIDSet)


12
{'d1mh1a_', 'd1dkia_', 'd1deua_', 'd1pvta_', 'd1eu3a1', 'd1nova_', 'd1bd7a_', 'd1pcia_', 'd4ax1b_', 'd1nria_', 'd1ngvb_', 'd3msza1'}


In [4]:
validIDSet = IDSet - errorIDSet
print(len(validIDSet))

14262


In [6]:
# 筛选得到 valid_pair
file_name = "/home/wngys/scop/TMalign/tmalign40.txt"
out_file = "/home/wngys/lab/DeepFold/pair/valid_pair.txt"
f_out = open(out_file, "w")

with open(file_name, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        id1 = lines.split('\t')[0]
        id2 = lines.split('\t')[1]

        if id1 in validIDSet and id2 in validIDSet:
            f_out.write(lines)
f_out.close()

In [17]:
# 根据valid_pair文件，将align文件左侧一列 按蛋白质类别 分成许多txt
file_name = "/home/wngys/lab/DeepFold/pair/valid_pair.txt"
dir_name = "/home/wngys/lab/DeepFold/pair/pair_score/"
dic = {}

with open(file_name, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        id1 = lines.split('\t')[0]
        id2 = lines.split('\t')[1]
        score = lines.split('\t')[2].split('\n')[0]
        if id1 not in dic:
            dic[id1] = []
        dic[id1].append((id2, score))

for id in dic.keys():
    f_name = dir_name + id + ".txt"
    f_out = open(f_name, "w")
    for ele in dic[id]:
        f_out.write(ele[0])
        f_out.write('\t')
        f_out.write(ele[1])
        f_out.write("\n")
    f_out.close()

In [1]:
# 得到valid_set集合(align文件第一列蛋白质集合)
file_name = "/home/wngys/lab/DeepFold/pair/valid_pair.txt"
valid_set = set()
with open(file_name, "r") as f_r:
    while True:
        lines = f_r.readline()
        if not lines:
            break
        id1 = lines.split('\t')[0]
        id2 = lines.split('\t')[1]
        valid_set.add(id1)

print(len(valid_set))

14244


In [2]:
# 验证一下Pair文件夹 文件数目是否等于align文件左侧蛋白质集合 元素数目
dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90"
files_n = os.listdir(dir)
print(len(files_n))

14244


In [5]:
import numpy as np

np.save("/home/wngys/lab/DeepFold/protein_infor/leftIDArray.npy", list(valid_set))

In [6]:
ll = np.load("/home/wngys/lab/DeepFold/protein_infor/leftIDArray.npy", allow_pickle=True).tolist()
print(len(ll))

14244


In [21]:
# 由Pair文件 将第二列TM-score转换为 bool 表示 positive or negtive
import os
dir_pair = "/home/wngys/lab/DeepFold/pair/pair_score"
out_dir = "/home/wngys/lab/DeepFold/pair/pair_bool_90/"
p = 0.9
i = 0
for file_n in os.listdir(dir_pair):
    # if i >= 5:
    #     break
    file_name = os.path.join(dir_pair, file_n)
    Max_score = -1
    with open(file_name, "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            # id2 = lines.split('\t')[0]
            score = lines.split('\t')[1].split("\n")[0]
            score = float(score)
            Max_score = max(score, Max_score)

    threshold = Max_score *p
    # print(file_n)
    # print(f"Max_score:{Max_score}")
    f_w = open(out_dir + file_n, "w")

    with open(file_name, "r") as f_r:
        while True:
            lines = f_r.readline()
            if not lines:
                break
            id2 = lines.split('\t')[0]
            score = lines.split('\t')[1].split("\n")[0]
            score = float(score)
            f_w.write(id2)
            f_w.write("\t")

            if score >= threshold:
                # print(id2, score)
                f_w.write(str(1))
                f_w.write("\n")
            else:
                f_w.write(str(0))
                f_w.write("\n")
    f_w.close()